In [17]:
# === Imports and Globals ===
import numpy as np
from pettingzoo.utils import aec_to_parallel
from mpe2 import simple_spread_v3
from failure_api.communication_models import ProbabilisticModel
from failure_api.wrappers import CommunicationWrapper


In [18]:
# === Basic Wrapping Test ===
N = 3
max_cycles = 25
failure_prob = 0.5

base_env = simple_spread_v3.env(N=N, max_cycles=max_cycles)
agent_ids = base_env.possible_agents

failure_model = ProbabilisticModel(agent_ids=agent_ids, failure_prob=failure_prob)
env = CommunicationWrapper(base_env, failure_models=[failure_model])

print("✅ Environment wrapped successfully.")


✅ Environment wrapped successfully.


In [19]:
# === Agent Cycling Check ===
env.reset()
visited_agents = set()

for _ in range(len(agent_ids) * 2):  # Step through 2 full cycles
    agent = env.agent_selection
    visited_agents.add(agent)
    action = env.action_space(agent).sample()
    env.step(action)

assert visited_agents == set(agent_ids), f"❌ Missing agents in cycle: {set(agent_ids) - visited_agents}"
print("✅ All agents participated in stepping.")


✅ All agents participated in stepping.


In [20]:
# === Parallel Conversion Check ===
parallel_env = aec_to_parallel(env)
obs, _ = parallel_env.reset()
agent = agent_ids[0]

assert isinstance(obs[agent], (np.ndarray, dict)), "❌ Observation format invalid after wrapping"
print("✅ AEC to Parallel conversion successful. Observation format is valid.")


✅ AEC to Parallel conversion successful. Observation format is valid.


In [21]:
# === RL Agent I/O Consistency Check ===
obs, _ = parallel_env.reset()
agent = agent_ids[0]
obs_sample = obs[agent]
act_sample = parallel_env.action_space(agent).sample()

print("🔎 Observation sample:", obs_sample)
print("🔎 Action sample:", act_sample)

assert obs_sample is not None, "❌ Observation is None!"
assert isinstance(act_sample, (int, np.integer)), "❌ Action is not a valid integer!"
print("✅ Observations and actions are RL-compatible.")


🔎 Observation sample: {'agent_0': array([ 0.        ,  0.        ,  0.62890613, -0.33670315, -1.2189927 ,
        1.022272  , -0.89130807,  0.22785118, -1.004614  ,  0.8011287 ,
       -1.1653311 ,  0.18965533, -0.3012079 ,  0.02562328,  0.        ,
        0.        ,  0.        ,  0.        ], dtype=float32)}
🔎 Action sample: 0
✅ Observations and actions are RL-compatible.


In [22]:
# === RL Agent I/O Consistency Check ===


obs, _ = parallel_env.reset()
agent = agent_ids[0]
obs_sample = obs[agent]
act_sample = parallel_env.action_space(agent).sample()

print("🔎 Observation sample:", obs_sample)
print("🔎 Action sample:", act_sample)

assert obs_sample is not None, "❌ Observation is None!"
assert isinstance(act_sample, (int, np.integer)), "❌ Action is not a valid integer!"
print("✅ Observations and actions are RL-compatible.")


🔎 Observation sample: {'agent_0': array([ 0.        ,  0.        ,  0.93148357, -0.43511134, -0.64171386,
        1.1667687 , -0.6242568 ,  0.5788818 , -0.35251883,  0.37126014,
       -1.7695892 ,  1.397471  , -0.96134335, -0.2914375 ,  0.        ,
        0.        ,  0.        ,  0.        ], dtype=float32)}
🔎 Action sample: 2
✅ Observations and actions are RL-compatible.


In [29]:
# === FailureAPI Behavior Check (ParallelEnv-Compatible) ===

aec_env = env  # Your CommunicationWrapper (AEC)
parallel_env = aec_to_parallel(aec_env)

obs, _ = parallel_env.reset()
comm_matrix = aec_env._comm_matrix

print("📡 Communication Matrix:")
for sender in agent_ids:
    row = {receiver: comm_matrix[sender][receiver] for receiver in agent_ids}
    print(f"{sender}: {row}")

# Show initial observation to check masking
for agent in agent_ids:
    print(f"🔎 Initial observation for {agent}:", obs[agent])


AttributeError: accessing private attribute '_comm_matrix' is prohibited

In [24]:
# === FailureAPI Behavior Check ===
parallel_env.reset()
comm_matrix = env.communication_matrix  # Direct access for testing

print("📡 Communication Matrix:")
print(comm_matrix)

obs = parallel_env.observe(agent)
print(f"🔎 Observation for {agent}:", obs[agent])

if not comm_matrix[agent][agent_ids[1]]:  # Example mask
    assert np.allclose(obs[agent_ids[1]], 0), f"❌ Observation not masked for {agent_ids[1]}"
    print(f"✅ Masking applied to {agent_ids[1]} as expected.")
else:
    print(f"ℹ️ Communication allowed with {agent_ids[1]}")


SyntaxError: invalid syntax (3237731005.py, line 3)

In [25]:
# === FailureAPI Behavior Check ===
parallel_env.reset()
comm_matrix = env.communication_matrix  # Direct access for testing

print("📡 Communication Matrix:")
print(comm_matrix)

obs = parallel_env.observe(agent)
print(f"🔎 Observation for {agent}:", obs[agent])

if not comm_matrix[agent][agent_ids[1]]:  # Example mask
    assert np.allclose(obs[agent_ids[1]], 0), f"❌ Observation not masked for {agent_ids[1]}"
    print(f"✅ Masking applied to {agent_ids[1]} as expected.")
else:
    print(f"ℹ️ Communication allowed with {agent_ids[1]}")


SyntaxError: invalid syntax (3237731005.py, line 3)